# Crypto Sentiment – prise en main

Notebook démo pour interroger l'API locale (FastAPI) et récupérer les résultats d'analyse de sentiment.

## Prérequis
- API lancée en local (FastAPI) via Poetry.
- Kernel du notebook sur le venv `.venv` (Python 3.11) pour utiliser les mêmes libs que l'API.
- Selenium uniquement si tu passes `method="selenium"` (driver nécessaire).

In [ ]:
API_URL = "http://127.0.0.1:8000"  # Assure-toi que l'API tourne (uvicorn app.main:app --reload)

In [ ]:
import requests
import pandas as pd
import plotly.express as px

print("requests", requests.__version__)
print("pandas", pd.__version__)

## 1. Démarrer l'API

Terminal (venv Poetry) :
```
export PATH="$HOME/Library/Python/3.14/bin:$PATH"
POETRY_VIRTUALENVS_IN_PROJECT=1 poetry run uvicorn app.main:app --reload
```
Endpoints utiles : `GET /health`, `GET /cryptos`, `GET /prices`, `POST /scrape`, `POST /analyze`, `POST /analyze/multi`.

In [ ]:
# Statut de l'API
resp = requests.get(f"{API_URL}/health", timeout=10)
resp.raise_for_status()
resp.json()

In [ ]:
# Cryptos et subreddits connus par le scraper HTTP
resp = requests.get(f"{API_URL}/cryptos", timeout=20)
resp.raise_for_status()
data = resp.json()
pd.DataFrame({"crypto": data.get("cryptos", []), "subreddit": data.get("subreddits", [])})

## 2. Analyse simple (HTTP scraper)
L'API mappe automatiquement la crypto vers le subreddit connu (voir `/cryptos`).

In [ ]:
analyze_payload = {
    "crypto": "bitcoin",
    "subreddit": "Bitcoin",  # peut être ignoré si crypto connue
    "limit": 20,
    "method": "http",
}

resp = requests.post(f"{API_URL}/analyze", json=analyze_payload, timeout=120)
resp.raise_for_status()
result = resp.json()

summary = {
    "posts_scraped": result.get("posts_scraped"),
    "posts_analyzed": result.get("posts_analyzed"),
    "avg_sentiment": result.get("sentiment", {}).get("average"),
    "distribution": result.get("sentiment", {}).get("distribution"),
    "price": result.get("price"),
}
summary

In [ ]:
# Aperçu des posts + scores
posts_table = pd.DataFrame(result.get("posts", []))
posts_table.head(10)

## 3. Analyse multi-crypto
Interroge `/analyze/multi` pour récupérer plusieurs cryptos en une fois.

In [ ]:
multi_payload = {
    "cryptos": ["bitcoin", "ethereum", "solana"],
    "limit_per_crypto": 20,
}

resp = requests.post(f"{API_URL}/analyze/multi", json=multi_payload, timeout=240)
resp.raise_for_status()
multi = resp.json()

rows = []
for name, data in multi.items():
    distrib = data.get("sentiment", {}).get("distribution", {})
    rows.append({
        "crypto": name,
        "posts_analyzed": data.get("posts_analyzed"),
        "avg_sentiment": data.get("sentiment", {}).get("average"),
        "pos": distrib.get("positive"),
        "neu": distrib.get("neutral"),
        "neg": distrib.get("negative"),
    })
pd.DataFrame(rows)

## 4. Scraping seul
Endpoint `/scrape` : permet d'inspecter les posts sans lancer le modèle de sentiment.

In [ ]:
scrape_payload = {
    "crypto": "bitcoin",
    "subreddit": "Bitcoin",
    "limit": 10,
    "method": "http",
}

resp = requests.post(f"{API_URL}/scrape", json=scrape_payload, timeout=60)
resp.raise_for_status()
posts = resp.json().get("posts", [])
pd.DataFrame(posts)[["title", "score", "url"]].head(5)

## 5. Prix spot
Endpoints `/prices` et `/prices/{crypto}`.

In [ ]:
resp = requests.get(f"{API_URL}/prices", timeout=20)
resp.raise_for_status()
prices = resp.json()
pd.DataFrame(prices)

In [ ]:
resp = requests.get(f"{API_URL}/prices/bitcoin", timeout=20)
resp.raise_for_status()
resp.json()

## 6. Visualisation rapide
Histogramme des labels pour l'analyse simple précédente.

In [ ]:
labels = result.get("sentiment", {}).get("distribution", {})
labels_df = pd.DataFrame({
    "label": list(labels.keys()),
    "count": list(labels.values()),
})
fig = px.bar(labels_df, x="label", y="count", title="Distribution des labels")
fig.show()

## 7. (Optionnel) Mode Selenium
Si un driver est dispo sur la machine :
```
payload = {"crypto": "bitcoin", "subreddit": "Bitcoin", "limit": 10, "method": "selenium"}
requests.post(f"{API_URL}/analyze", json=payload, timeout=180)
```
Le mode headless est activé côté API, mais nécessite un driver Chrome/Firefox configuré.